### 수치 미분과 수치 미분의 문제점

**미분이란?**  
변화율을 의미하며 물체의 시간에 따른 위치 변화율은 속도가 됨. 

**수치 미분 구현**  
(컴퓨터로 극한을 표현하기 위해 1e-4와 같은 값을 이용하였고 이러한 미세한 차이를 이용하여 함수의 변화량을 구하는 방법을 수치미분이라고 함)

In [7]:
#y=x**2 미분

import numpy as np

class Variable:
    def __init__(self,data):
        self.data=data

class Function:
    def __call__(self,input):
        x=input.data
        y=self.forward(x)
        output=Variable(y)
        return output

class Square(Function):
    def forward(self,x):
        return x**2

def numerical_diff(f,x,eps=1e-4):
    x0=Variable(x.data-eps)
    x1=Variable(x.data+eps)
    y0=f(x0)
    y1=f(x1)
    return (y1.data-y0.data)/(2*eps)

f=Square()
x=Variable(np.array(2.0))
dy=numerical_diff(f,x)
print(dy)

4.000000000004


In [9]:
#y=((e^x)^2)^2

class Exp(Function):
    def forward(self,x):
        return np.exp(x)
    
def f(x):
    A=Square()
    B=Exp()
    C=Square()
    return C(B(A(x)))

x=Variable(np.array(0.5))
dy=numerical_diff(f,x)
print(dy)

3.2974426293330694


수치미분은 계산에 따라 다르지만 오차를 포함하고 변수가 여러 개인 계산을 미분한다면 변수 각각을 미분해야하기 때문에 신경망에 적용하는건 현실적이지 못하다는 단점이 있음.  

**※계산비용과 정확도 면에서 문제가 존재**

# 역전파(backpropagation)

**연쇄법칙(chain rule)**: 함성 함수의 미분은 구성 함수를 각각 미분하고 곱한 것과 같다.  
  
y=F(x)라는 함수는 a=A(x), b=B(a), y=C(b)라는 세 함수로 구성되어 있다고 가정
<img src="image/합성함수의예.png" width="800" height="300">   
이때 x에 대한 y의 미분은 아래의 식으로 표현이 가능함  
$\frac{dy}{dx}=((\frac{dy}{dy}\frac{dy}{db})\frac{db}{da})\frac{da}{dx}$  
<img src="image/역전파그래프.png" width="800" height="300">   

#### 역전파를 이용한 미분 구현

In [15]:
class Variable:
    def __init__(self,data):
        self.data=data
        self.grad=None
        
class Function:
    def __call__(self,input):
        x=input.data
        y=self.forward(x)
        output=Variable(y)
        self.input=input
        return output
    
def forward(self,x):
    raise NotImplementedError()
    
def backward(self,gy):
    raise NotImplementedError()
    
class Square(Function):
    def forward(self,x):
        y=x**2
        return y
    
    def backward(self,gy):
        x=self.input.data
        gx=2*x*gy
        return gx
    
class Exp(Function):
    def forward(self,x):
        y=np.exp(x)
        return y
    
    def backward(self,gy):
        x=self.input.data
        gx=np.exp(x)*gy
        return gx
    
A=Square()
B=Exp()
C=Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)

y.grad=np.array(1.0)   #역전파는 dy/dy=1에서 시작함  
b.grad=C.backward(y.grad)
a.grad=B.backward(b.grad)
x.grad=A.backward(a.grad)
print(x.grad)

3.297442541400256
